In [ ]:
# summa oru text

In [1]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as scs
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
#from arch import arch_model
from arch.univariate import HARX
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ValueWarning)


In [3]:
df=pd.read_csv('Datasets\AAPL_20200102.csv.gz', compression='gzip')
df.head()

# =============================================================================
# Data cleaning
# =============================================================================
# Remove entries with zero or negative bid, ask, transanction price or size



<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\prvij\AppData\Local\Temp\ipykernel_9484\491745702.py:1: SyntaxWarning: invalid escape sequence '\A'
  df=pd.read_csv('Datasets\AAPL_20200102.csv.gz', compression='gzip')


,DATE,TIME_M,SYM_ROOT,EX,SIZE,PRICE,NBO,NBB,NBOqty,NBBqty,midpoint,BuySellLR,BuySellEMO,BuySellCLNV
0,02JAN2020,34200.134554,AAPL,K,250.0,296.24,296.29,296.24,100.0,200.0,296.265,-1.0,-1.0,-1.0
1,02JAN2020,34200.134577,AAPL,K,50.0,296.24,296.29,296.21,100.0,100.0,296.250,-1.0,NaN,NaN
2,02JAN2020,34200.134742,AAPL,K,108.0,296.21,296.29,296.21,100.0,100.0,296.250,-1.0,-1.0,-1.0
3,02JAN2020,34200.157564,AAPL,P,1.0,296.28,296.29,296.10,200.0,100.0,296.195,1.0,1.0,1.0
4,02JAN2020,34200.157564,AAPL,P,1.0,296.28,296.29,296.10,200.0,100.0,296.195,1.0,1.0,1.0


In [5]:
df_out = (df['NBB'] <= 0) | (df['NBBqty'] <= 0) |  (df['NBO'] <= 0) | (df['NBOqty'] <= 0) | \
    (df['PRICE'] <= 0) | (df['SIZE'] <= 0)

In [6]:
df_out

0         False
1         False
2         False
3         False
4         False
          ...  
273629    False
273630    False
273631    False
273632    False
273633    False
Length: 273634, dtype: bool

In [7]:
df = df[~df_out].reset_index(drop=True)

In [8]:
len(df)

273634

In [9]:
df['Stamp'] = pd.to_datetime(df['DATE']) + df['TIME_M'].apply(lambda x: pd.Timedelta(x,unit='sec'))
df.set_index('Stamp', drop=True, inplace = True)


In [10]:
df

,DATE,TIME_M,SYM_ROOT,EX,SIZE,PRICE,NBO,NBB,NBOqty,NBBqty,midpoint,BuySellLR,BuySellEMO,BuySellCLNV
Stamp,,,,,,,,,,,,,,
2020-01-02 09:30:00.134553762,02JAN2020,34200.134554,AAPL,K,250.0,296.2400,296.29,296.24,100.0,200.0,296.265,-1.0,-1.0,-1.0
2020-01-02 09:30:00.134577180,02JAN2020,34200.134577,AAPL,K,50.0,296.2400,296.29,296.21,100.0,100.0,296.250,-1.0,NaN,NaN
2020-01-02 09:30:00.134742137,02JAN2020,34200.134742,AAPL,K,108.0,296.2100,296.29,296.21,100.0,100.0,296.250,-1.0,-1.0,-1.0
2020-01-02 09:30:00.157563825,02JAN2020,34200.157564,AAPL,P,1.0,296.2800,296.29,296.10,200.0,100.0,296.195,1.0,1.0,1.0
2020-01-02 09:30:00.157564014,02JAN2020,34200.157564,AAPL,P,1.0,296.2800,296.29,296.10,200.0,100.0,296.195,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-02 15:59:59.944672526,02JAN2020,57599.944673,AAPL,Q,4370.0,300.6000,300.60,300.58,6700.0,6800.0,300.590,1.0,1.0,1.0
2020-01-02 15:59:59.963696249,02JAN2020,57599.963696,AAPL,P,200.0,300.6000,300.60,300.58,2300.0,6900.0,300.590,1.0,1.0,1.0
2020-01-02 15:59:59.984673484,02JAN2020,57599.984673,AAPL,D,500.0,300.5900,300.60,300.58,2300.0,7100.0,300.590,-1.0,-1.0,-1.0


In [3]:
''
### Import merged data from TAQ for AAPL on 02/01/2020
df=pd.read_csv('AAPL_20200102.csv.gz', compression='gzip')
df.head()

# =============================================================================
# Data cleaning
# =============================================================================
# Remove entries with zero or negative bid, ask, transanction price or size
df_out = (df['NBB'] <= 0) | (df['NBBqty'] <= 0) |  (df['NBO'] <= 0) | (df['NBOqty'] <= 0) | \
    (df['PRICE'] <= 0) | (df['SIZE'] <= 0)
df = df[~df_out].reset_index(drop=True)

# Remove entries with negative bid-ask spread
df['SPREAD'] = df['NBO'] - df['NBB']
df = df[~(df['SPREAD'] <= 0)].reset_index(drop=True)

# Check if there are duplicated entries
df['TIME_M'].duplicated(keep=False).sum()

# Remove entries for which the spread is more that 50 times the median spread on that day
medspread = df['SPREAD'].median()
df = df[~(df['SPREAD'] > 50*medspread)].reset_index(drop=True)

# Create a timestamp variable and set it as the index
df['Stamp'] = pd.to_datetime(df['DATE']) + df['TIME_M'].apply(lambda x: pd.Timedelta(x,unit='sec'))
df.set_index('Stamp', drop=True, inplace = True)

# =============================================================================
# Descriptive analysis
# =============================================================================
## Time series plot
df[['PRICE','NBO','NBB','midpoint']].plot(figsize=(6,4))

## Need to do filtering T4 also
## Delete entries with prices that are above the `ask' plus the bid–ask spread. 
## Similar for entries with prices below the `bid' minus the bid–ask spread
df_out = (df['PRICE'] > df['NBO'] + df['SPREAD']) | (df['PRICE'] < df['NBB'] - df['SPREAD'])
df = df[~df_out]
df[['PRICE','NBO','NBB','midpoint']].plot(figsize=(6,4))

## Bid-ask spread
df['SPREAD'].plot(figsize=(6,4),title='Bid-ask Spread')
df.groupby('BuySellLR')['SPREAD'].describe()

## Trading volumes
df['SIZE'].plot(figsize=(6,4),title='Trading Volume') # There is a big trade at early of the day

df['SIZE'].max() # 424836.0
tmp = df.loc[df['SIZE'] == df['SIZE'].max()]

df[df['SIZE'] != df['SIZE'].max()]['SIZE'].plot(figsize=(6,4),title='Trading Volume')

df.groupby('BuySellLR')['SIZE'].describe()
df[df['SIZE'] != df['SIZE'].max()].groupby('BuySellLR')['SIZE'].describe()

## Tick-by-tick returns
df['Return'] = np.log(df['PRICE']/df['PRICE'].shift(1))
df['Return'].plot(figsize=(6,4),title='Return')
df.groupby('BuySellLR')['Return'].describe()


'''
Computing Realized Volatility
'''

# =============================================================================
# Computing 5-min RV
# =============================================================================
### Method 1: Sampling log prices every 5 mins
lprice = np.log(df['PRICE'])

### First, need to sample log prices every 5 minutes
lprice_5m = lprice.resample(rule='5min',closed ='right',label ='right').apply('last')

### Compute log returns
ret_5m = lprice_5m.diff()
ret_5m.iloc[0] = lprice_5m.iloc[0] - lprice.iloc[0]

### 5-min RV
RV_5m = sum(ret_5m**2)


### Method 2: Sampling returns every 5 mins
returns = df['Return'].dropna()
returns_5m = returns.resample(rule='5min',closed ='right',label ='right').apply('sum')
RV_5mb = sum(returns_5m**2)

# =============================================================================
# Computing RV using different sampling frequencies
# =============================================================================
def RV(ret,freq='5min'):
    # ret: pandas series of intraday log returns with datetime index
    # freq: string - frequency to be used
    
    ret.dropna(inplace=True)
    ret_freq = ret.resample(rule=freq,closed ='right',label ='right').apply('sum')
    return sum(ret_freq**2)


freqs = ['1s', '5s', '10s', '15s', '30s',
         '1min', '2min', '5min', '10min', '15min', '20min', '30min']
RVs = {freq: RV(returns,freq) for freq in freqs}
RVs['tick'] = sum(returns**2)
# Reorder frequencies
RVs = {freq: RVs[freq] for freq in ['tick']+freqs} 
# Convert RVs to data frame
RVs = pd.Series(RVs)
# Scale/multiply RVs by 10000
RVs = RVs*10000
RVs

FileNotFoundError: [Errno 2] No such file or directory: 'AAPL_20200102.csv.gz'

In [2]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as scs
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
#from arch import arch_model
from arch.univariate import HARX
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ValueWarning)
